# TRAVELING SALESMAN: SIMULATED ANNEALING
Simulated annealing (SA) is a probabilistic technique for approximating the global optimum of a given function. Specifically, it is a metaheuristic to approximate global optimization in a large search space. It is often used when the search space is discrete (e.g., all tours that visit a given set of cities).

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math 
from math import floor
from random import randint
import csv as csv
#to shuffle dataframe
from sklearn.utils import shuffle 
from IPython.display import display, HTML
import scipy 
from scipy.misc import comb # comb(n,k, exact=True)
from math import inf
from math import exp, expm1
import decimal
import random
# from scipy import special
# from scipy.special import comb
CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [2]:
dist_mat = 'distance_matrix'
def read_data(fileName):
    df = pd.read_csv(fileName)
    return df
    
def check_packaging(df):
    rows, cols = df.shape #size of the data set
    return (rows, cols)

def data_check(df, n=3):
    df_top_n = df.head(n)
    return (df_top_n)

def check_ns(df):
    ns = df.describe()
    return ns

In [3]:
df_distMat = read_data('%s.csv'%dist_mat)
print("rows(%s) x cols(%s) "%check_packaging(df_distMat))
print()
print("%s"%data_check(df_distMat))
print()
print(check_ns(df_distMat))
print()
num_cities = df_distMat.shape[0]
# df_cityDist.set_index('city')
print('Number of cities including depot: ', num_cities)

rows(6) x cols(6) 

      0     1     2     3     4     5
0   NaN  36.0  32.0  54.0  20.0  40.0
1  36.0   NaN  22.0  58.0  54.0  67.0
2  32.0  22.0   NaN  36.0  42.0  71.0

              0          1          2          3          4         5
count   5.00000   5.000000   5.000000   5.000000   5.000000   5.00000
mean   36.40000  47.400000  40.600000  58.000000  42.200000  63.00000
std    12.36123  18.132843  18.487834  20.736441  13.236314  21.05944
min    20.00000  22.000000  22.000000  36.000000  20.000000  40.00000
25%    32.00000  36.000000  32.000000  50.000000  42.000000  45.00000
50%    36.00000  54.000000  36.000000  54.000000  45.000000  67.00000
75%    40.00000  58.000000  42.000000  58.000000  50.000000  71.00000
max    54.00000  67.000000  71.000000  92.000000  54.000000  92.00000

Number of cities including depot:  6


In [4]:
df_distMat.fillna(inf,inplace = True)
df_distMat
# df_distMat.iloc[1,0]

,0,1,2,3,4,5
0,inf,36.000000,32.000000,54.000000,20.000000,40.000000
1,36.000000,inf,22.000000,58.000000,54.000000,67.000000
2,32.000000,22.000000,inf,36.000000,42.000000,71.000000
3,54.000000,58.000000,36.000000,inf,50.000000,92.000000
4,20.000000,54.000000,42.000000,50.000000,inf,45.000000
5,40.000000,67.000000,71.000000,92.000000,45.000000,inf


In [5]:
# # list1 = list(range(0,10))
# # arr = np.asarray(list1)
# # type(arr[0])
# # # arr = np.array([np.random.permutation(range(10))])
arr = np.random.permutation(range(10))
display(arr)
arr1 = np.random.permutation(range(10))
display(arr1)
arr[3] = arr1[4]
display(arr)
b = arr
display(b[-1])
# n = [0]
# arr = np.array([arr,arr[0])
# a = 10
# b = 11
# if a!=b:
#     print('nope')
# else:
#     print('yay')
# len(arr)

# d = decimal.Decimal(random.randrange(100))/100
# print(d)
# a = d*5
# print(a)
# print(int(a))
# print(d*5)
# int(d*5)
# d = int(5*(random.randrange(100)/100))
# d
# a = exp(-(5+3)/2)
# a
# int(5.999)

array([6, 5, 1, 8, 0, 4, 3, 7, 9, 2])

array([5, 2, 9, 6, 7, 4, 3, 1, 0, 8])

array([6, 5, 1, 7, 0, 4, 3, 7, 9, 2])

2

In [26]:
arr = np.random.permutation(range(10))
myarray = np.zeros(10)
myarray = np.array(set(arr + arr[0]))
display(arr)
display(myarray)


array([2, 1, 8, 3, 4, 0, 5, 7, 9, 6])

array({2, 3, 4, 5, 6, 7, 8, 9, 10, 11}, dtype=object)

Note: The code below has merely reporoduced been reproduced in Python, programming language, with slight adaptations from Mr Woolway's SA code.

In [7]:
#Helper function. Simply computes a distance of a proposed path
 
# INPUTS: 
#          distMatrix   - the distance matrix: pandas dataframe structure
#          proposedPath - the sample path to compute : numpy array
#
# OUTPUTS: 
#          totaldist    - the corresponding distance of the proposed path

In [8]:
def distCalculatorReturn(distMatrix, proposedPath):
    dist = 0
    i = 0
    while i!= proposedPath.shape[0]-1:
        dist = dist + distMatrix.iloc[proposedPath[i], proposedPath[i+1]]
        i = i+1
    dist = dist + distMatrix.iloc[0,proposedPath[0]]#add distance from depot to first state
    dist = dist + distMatrix.iloc[proposedPath[-1],0]#add distance from last state to depot
    totaldist = dist
    return totaldist

In [36]:
#INPUTS: 
#D    - some given distance matrix (symmetric and 0's on diag)
#n    - the number of cities to vist

#OUTPUTS:
#path - the optimal path according to the simulated annealing
#dist - the corresponding distance to the optimal path
#distCal - the distance of the path calculated by the distance matrix
def simulates_annealing(D, number_states):  
    n = number_states-1
    T = 10000 #Initial temperature
    L = 20*n #Length of the markov chain
    e = (1*exp(1))**(-9) #Tolerance
    
    
    x  = np.random.permutation(range(1,number_states)) # creates an array of all the states besides the depot in a random order
    fx = distCalculatorReturn(D, x) #calculates the distance of the random path
    
    while T > e:
        for i in range(1,L):
            rand1 = decimal.Decimal(random.randrange(100))/100
            rand2 = decimal.Decimal(random.randrange(100))/100
            num1 = int(rand1*n)
            num2 = int(rand2*n)
            
            #ensuring num1 and num2 aren't equal
            while num1==num2:
                rand3 = decimal.Decimal(random.randrange(100))/100
                num1 = int(rand3*n)
                
            y = x # copy of the random path and making random changes to compare two paths
            swap1 = y[num1]
            y[num1] = y[num2]
            y[num2] = swap1
            fy = distCalculatorReturn(D,y)
            
            if fy < fx:
                x = y
                fx = fy
            elif decimal.Decimal(random.randrange(100))/100 < exp(1)**(-(fy-fx)/T):
                x = y
                fx = fy
            
        T=0.9*T
    path = x
    dist = fx 
    dist_calc_path = distCalculatorReturn(D,x)

    return path, dist, dist_calc_path

In [37]:
path, dist, distCalc = simulates_annealing(df_distMat, num_cities)
display(path)
display(dist)
display(distCalc)

array([1, 4, 3, 5, 2])

229.0

335.0

In [38]:
df_distMat

,0,1,2,3,4,5
0,inf,36.000000,32.000000,54.000000,20.000000,40.000000
1,36.000000,inf,22.000000,58.000000,54.000000,67.000000
2,32.000000,22.000000,inf,36.000000,42.000000,71.000000
3,54.000000,58.000000,36.000000,inf,50.000000,92.000000
4,20.000000,54.000000,42.000000,50.000000,inf,45.000000
5,40.000000,67.000000,71.000000,92.000000,45.000000,inf


In [39]:
D = df_distMat
display(df_distMat)
number_states = df_distMat.shape[0]
n = number_states-1
T = 10000 #Initial temperature
L = 20*n #Length of the markov chain
e = (1*exp(1))**(-9) #Tolerance
x  = np.random.permutation(range(1,number_states)) # creates an array of all the states besides the depot in a random order
display(x)
fx = distCalculatorReturn(D, x) #calculates the distance of the random path
# print(fx)
####CHECK FROM HERE:
#The proposed path and the distance calculated do not correlate.
#The minimum distance is returned but the correct path is not returned
while T > e:
    for i in range(1,L):
        rand1 = decimal.Decimal(random.randrange(100))/100
        rand2 = decimal.Decimal(random.randrange(100))/100
        num1 = int(rand1*n)
        num2 = int(rand2*n)
        while num1==num2:
            rand3 = decimal.Decimal(random.randrange(100))/100
            num1 = int(rand3*n)
        y = x
        swap1 = y[num1]
        y[num1] = y[num2]
        y[num2] = swap1

        fy = distCalculatorReturn(D,y)
        if fy < fx:
            x = y
            fx = fy
        elif decimal.Decimal(random.randrange(100))/100 < exp(1)**(-(fy-fx)/T):
            x = y
            fx = fy
    T=0.9*T
proposed_path = x
dist_proposed_path = fx  


,0,1,2,3,4,5
0,inf,36.000000,32.000000,54.000000,20.000000,40.000000
1,36.000000,inf,22.000000,58.000000,54.000000,67.000000
2,32.000000,22.000000,inf,36.000000,42.000000,71.000000
3,54.000000,58.000000,36.000000,inf,50.000000,92.000000
4,20.000000,54.000000,42.000000,50.000000,inf,45.000000
5,40.000000,67.000000,71.000000,92.000000,45.000000,inf


array([1, 3, 5, 2, 4])

In [40]:
proposed_path

array([2, 4, 1, 5, 3])

In [41]:
dist_proposed_path

229.0

In [42]:
dist = distCalculatorReturn(df_distMat,proposed_path)
print(dist)


341.0
